C++ presentation of 'Templet application' interface and model
=============================================================

```
 -------------------------------------
 TEMPLET_APP commands |  CLI commands 
 ---------------------|---------------
 write_event          |  event        
 read_events          |  log          
 reply_on_query       |  reply        
 write_query          |  query        
 read_answer          |  answer       

TOKEN(NAME,GROUP)
EVENT(ORDINAL,TAG,EXTERN,NAME,GROUP,DATA)
ANSWER(ORDINAL,NAME,GROUP,DATA)

write_event(TOKEN,TAG,DATA)   --> ORDINAL \/ false
read_events(TOKEN,ORDINAL)    --> EVENT[] \/ false
write_query(TOKEN,TAG,DATA)   --> ORDINAL \/ false
reply_on_query(TOKEN,ORDINAL,DATA) --> true \/ false
read_answer(TOKEN,ORDINAL)    --> DATA \/ false
```

In [2]:
#include <string>
#include <list>
#include <map>

using namespace std;

struct TEMPLET_APP{

public: // service interface    

typedef string   NAME;
typedef unsigned GROUP;
typedef unsigned ORDINAL;
typedef unsigned TAG;
typedef bool     EXTERN;

#define ASSERT(expr)  if(!(expr))return false;

bool write_event_permission(GROUP);
bool read_events_permission(GROUP);
bool read_answers_permission(GROUP);
bool reply_on_query_permission(GROUP);
bool client_permission(GROUP);

struct TOKEN{
    NAME  name;  // who is the token owner?
    GROUP group; // which group does the token owner belong to?
}; 

struct DATA{ long size; void* data; }; //abstract binary data

struct EVENT{
    ORDINAL ord;
    TAG     tag;
    EXTERN  ext;  // ext==true used for client queries
    NAME    name;
    GROUP   group;
    DATA    data; // if ext==true, data may include query GUID
};

struct ANSWER{
    ORDINAL ord;  // query ordinal number
    NAME    name; // who wrote the answer?
    GROUP   group;
    DATA    data;
};

/*-1-*/
bool write_event(TOKEN jwt,TAG t,DATA d,ORDINAL& ord){
   
    ASSERT(write_event_permission(jwt.group));

    // EVENT.ord is a key autoincrement field
    EVENT ev;   
    ev.ord = events.size()+1; 
    ev.tag = t;  ev.ext = false; // regular (internal) event
    ev.data = d; ev.name = jwt.name; ev.group = jwt.group;
    events.push_back(ev);
    
    return true;
}
    
/*-2-*/              
bool read_events(TOKEN jwt,ORDINAL ord,list<EVENT>& evs){
    
    ASSERT(read_events_permission(jwt.group));

    evs.clear(); 
    for(EVENT ev:events)
        if(ev.ord>=ord) evs.push_back(ev);

    return true;
}

/*-3-*/
bool write_query(TOKEN jwt,TAG t,DATA d,ORDINAL& ord){
    
    ASSERT(client_permission(jwt.group));
    
    EVENT ev;
    ev.ord = events.size()+1; 
    ev.tag = t;  ev.ext = true; // external event 
    ev.data = d; ev.name = jwt.name; ev.group = jwt.group;   
    events.push_back(ev);

    ord = ev.ord;
    return true;
}

/*-4-*/
bool reply_on_query(TOKEN jwt,ORDINAL ord,DATA d){

    ASSERT(reply_on_query_permission(jwt.group));

    // ANSWER.ord is a key field
    for(ANSWER ans:answers){
        if(ans.ord==ord) return true;
    }

    for(EVENT ev:events){
        if(ev.ord==ord && ev.ext){
            ANSWER ans;
            ans.ord = ord; ans.name = jwt.name; ans.group = jwt.group; ans.data = ev.data;
            answers.push_back(ans);
            return true;
        }
    }  
    return false;
}    

/*-5-*/            
bool read_answer(TOKEN jwt,ORDINAL ord,DATA& d){
   
    ASSERT(client_permission(jwt.group)||read_answers_permission(jwt.group));

    for(ANSWER ans:answers)
    for(EVENT  ev:events)
    if(ans.ord == ord && ev.ord==ord)
    {
        ASSERT(client_permission(jwt.group) ? ev.name==jwt.name : read_answers_permission(jwt.group));
        d = ans.data;
        return true;       
    }
    return false;
}

private: // service state
    
list<EVENT>  events;
list<ANSWER> answers;

};

```
Интерфейс параметров командной строки
=====================================

Идея: возможно реализовать сервис синхронизации глобального состояния на базе готовых систем удаленного запуска задач (Everest, HTCondor, Dirac и т.д.) для быстрого прототипирования и использования в качестве эталонной реализации для полнофункционального сервиса, а затем CLI-интерфейса полнофункционального сервиса.
Для этого реализуется консольное приложение с описанным ниже интерфейсом.

Параметры
=========
(--base | -b) <base> база данных состояния приложения или endpoint приложения
(--jwt  | -j) <JWT> токен с описанием прав доступа для агента с заданным именем
(--tag  | -t) <num> тэг события
(--ordinal | -о) <num> порядковый номер события

Форматы файлов и stdout
=======================
<file>     формат определяется в приложении
<base>     SQLite файл базы данных, хранящей состояние приложения, или endpoint для подключения серверу приложения
<logfile>  запись фрагмента последовательных событий обновления состояния, внутренний формат

Команды
=======
(--event | -e) <file>  запись события, параметры: base, tag, в stdout - порядковый номер события
(--log   | -l) <logfile> выборка из лога событий, начиная с заданного ordinal, параметры: base, ordinal
(--reply | -r) <file>  ответ на запрос с указанным номером ordinal, параметры: base, ordinal
(--query | -q) <file>  файл с данными запроса, параметры: base, tag, в stdout - порядковый номер события-запроса
(--answer| -a) <file>  файл с ответом на запрос, параметры: base, ordinal
```